In [32]:
# License: BSD
# Author: Sasank Chilamkurthy

from __future__ import print_function, division

import torch
import torch.nn as nn
import torch.optim as optim
from torch.optim import lr_scheduler
import torch.backends.cudnn as cudnn
import numpy as np
import torchvision
from torchvision import datasets, models, transforms
import matplotlib.pyplot as plt
import time
import os
import copy
from pathlib import Path
from tqdm.notebook import tqdm, trange
from time import sleep
from random import *

cudnn.benchmark = True
plt.ion()   # interactive mode

In [33]:
# Clear cuda memory
import torch
torch.cuda.empty_cache()

In [34]:
# Engaging GPU
# Device will determine whether to run the training on GPU or CPU.
device = torch.device('cuda' if torch.cuda.is_available else 'cpu')
torch.cuda.is_available()

True

In [35]:
device

device(type='cuda')

In [36]:
### Butterfly

In [37]:
outputs = Path('./outputs')
if not outputs.is_dir():
    outputs.mkdir()

In [38]:
from warnings import filterwarnings
filterwarnings('ignore')

In [39]:
# function for saving weights of trained model
def save_model(epochs, model, optimizer, criterion, name='model', descr=''):

    torch.save({
                'epoch': epochs,
                'model_state_dict': model.state_dict(),
                'optimizer_state_dict': optimizer.state_dict(),
                'loss': criterion,
                'descr': descr,
                }, f'outputs/{name}.pth')

# Data Preprocessing

In [40]:
# Define relevant variables for the ML task
batch_size = 16
num_classes = 7
learning_rate = 0.001
num_epochs = 2

In [41]:
train_trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(p=0.5),
    transforms.RandomVerticalFlip(p=0.5),
    transforms.GaussianBlur(kernel_size=(5, 9), sigma=(0.1, 5)),
    transforms.RandomRotation(degrees=(30, 70)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

valid_trans = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(
        mean=[0.5, 0.5, 0.5],
        std=[0.5, 0.5, 0.5]
    )
])

In [42]:
from torchvision.datasets import ImageFolder
train_folder = ImageFolder("D:\\FALL 2022\\MS_Thesis\\MASTER'S THESIS\\EdmCrack600\\Data_Structure(Annotated)", transform=train_trans , )
test_folder = ImageFolder("D:\\FALL 2022\\MS_Thesis\\MASTER'S THESIS\\EdmCrack600\\Data_Structure(Annotated)", transform=valid_trans , )

In [43]:
len(train_folder.classes)

7

In [44]:
train_folder.classes

['ALLIGATOR ( Fatigue plus Longitudinal Cracks on Wheel Paths',
 'BLEEDING_',
 'DELAMINATION',
 'LONGITUDINAL LANE JOINT CRACKING_',
 'LONGITUDINAL OUTSIDE OF THE WHEEL PATHS_',
 'Raveling',
 'Transverse Crack']

In [45]:
classes_name = train_folder.classes
classes_name

['ALLIGATOR ( Fatigue plus Longitudinal Cracks on Wheel Paths',
 'BLEEDING_',
 'DELAMINATION',
 'LONGITUDINAL LANE JOINT CRACKING_',
 'LONGITUDINAL OUTSIDE OF THE WHEEL PATHS_',
 'Raveling',
 'Transverse Crack']

In [46]:
# Splitting data on batches
train_loader = torch.utils.data.DataLoader(train_folder, shuffle=True, batch_size=batch_size)
test_loader = torch.utils.data.DataLoader(test_folder, shuffle=False, batch_size=batch_size)

In [47]:
# Let's take a look at the first batch
data, labels = next(iter(train_loader))
# data, labels = data.cpu(), labels.cpu() ## Solution: https://stackoverflow.com/questions/59013109/runtimeerror-input-type-torch-floattensor-and-weight-type-torch-cuda-floatte

In [48]:
data.size

<function Tensor.size>

In [49]:
data.shape

torch.Size([16, 3, 224, 224])

In [50]:
type(data)

torch.Tensor

1. The input of a Pytorch Neural Network is of type [BATCH_SIZE] * [CHANNEL_NUMBER] * [HEIGHT] * [WIDTH]

2. 1×3×32×32 meaning that you have 1 image with 3 channels (RGB) with height 32 and width 32

3. . The formular of convolution is ((W - F + 2P)/ S )+1 and ((H - F + 2P)/ S )+1

4. W =  WIDTH, F  = FILTER_SIZE, P = PADDIND, S = STRIDE

with our input 1×3×32×32 after applying conv1 W will be 28 and H will be 28 and also applying (2,2) pooling halves the WIDTH and HEIGTH and We have 6 feature maps. So after the first con2d and pooling we end up with an image of dimension
1 * 6 * 14 * 14. Similaly for the second conv2d and pooling we will end up with an image of dimension 1 * 16 * 5 * 5 . Finally since we need a column vector for the first fc layer we should unroll our vector which is 16×5×5 = 400

https://discuss.pytorch.org/t/input-size-of-fc-layer-in-tutorial/14644


## Here, the given relationship between the kernel size and padding is that padding = (kernel size - 1) / 2. Max pooling with kernel_size = stride = 2 will decrease the width/height by a factor of 2 (rounded down if input shape is not even).



## Load VGG 19 Pretrained

In [51]:
import torch
model = models.vgg19(pretrained=True).to(device)
model

VGG(
  (features): Sequential(
    (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace=True)
    (2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace=True)
    (4): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (5): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace=True)
    (7): Conv2d(128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace=True)
    (9): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
    (10): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace=True)
    (12): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace=True)
    (14): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace=True)
    (16): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padd

In [52]:
if torch.cuda.is_available():
    model.cuda()

In [53]:
model.classifier[6] = nn.Linear(4096, 7).to(device) # original model has outputs for 1000 classes. 
# But there are only 75 classes so we have to change output layer

In [54]:
# Freezing all layers except last 15
for param in list(model.parameters())[:-15]:
    param.requires_grad = False

In [55]:
## Defining model optimizer and loss function
loss_fn = nn.CrossEntropyLoss()
opt = optim.Adam(model.parameters(), lr=1e-4)

In [56]:
from torchsummary import summary
summary(model, (3, 224, 224))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1         [-1, 64, 224, 224]           1,792
              ReLU-2         [-1, 64, 224, 224]               0
            Conv2d-3         [-1, 64, 224, 224]          36,928
              ReLU-4         [-1, 64, 224, 224]               0
         MaxPool2d-5         [-1, 64, 112, 112]               0
            Conv2d-6        [-1, 128, 112, 112]          73,856
              ReLU-7        [-1, 128, 112, 112]               0
            Conv2d-8        [-1, 128, 112, 112]         147,584
              ReLU-9        [-1, 128, 112, 112]               0
        MaxPool2d-10          [-1, 128, 56, 56]               0
           Conv2d-11          [-1, 256, 56, 56]         295,168
             ReLU-12          [-1, 256, 56, 56]               0
           Conv2d-13          [-1, 256, 56, 56]         590,080
             ReLU-14          [-1, 256,

In [57]:
def mean(l: list):
    return sum(l) / len(l)

In [58]:
from sklearn.metrics import accuracy_score

In [59]:
def plot_losses_and_acc(train_losses, train_accuracies, valid_losses, valid_accuracies): 
    fig, axes = plt.subplots(1, 2, figsize=(15, 10))
    axes[0].plot(train_losses, label='train_losses')
    axes[0].plot(valid_losses, label='valid_losses')
    axes[0].set_title('Losses')
    axes[0].legend()
    
    axes[1].plot(train_accuracies, label='train_losses')
    axes[1].plot(valid_accuracies, label='valid_losses')
    axes[1].set_title('Accuracy')
    axes[1].legend()

In [60]:
def validate(model, valid_data, loss_fn):
    valid_losses, valid_accuracies = [], []
    model.eval()
    with torch.no_grad():
        for X_batch, y_batch in tqdm(valid_data, leave=False):
            X_batch, y_batch = X_batch.to(device).float(), y_batch.to(device).long()
            logits = model(X_batch)
            loss = loss_fn(logits, y_batch)
            valid_losses.append(loss.item())
            preds = torch.argmax(logits, axis=1)
            
            valid_accuracies.append(((preds == y_batch).sum() / len(preds)).item())
    return mean(valid_losses), mean(valid_accuracies)
    

def train(model, train_data, valid_data, loss_fn, opt, epoches=5):
    train_losses, valid_losses = [], []
    train_accuracies, valid_accuracies = [], []
    
    for epoch in tqdm(range(epoches)):
        train_loss = []
        train_acc = []
        model.train()
        for X_batch, y_batch in tqdm(train_data, leave=False):
            opt.zero_grad()

            X_batch, y_batch = X_batch.to(device).float(), y_batch.to(device).long()
            logits = model(X_batch)
            loss = loss_fn(logits, y_batch,)
            train_loss.append(loss.item())

            pred = torch.argmax(logits, dim=1)
            train_acc.append(((pred == y_batch).sum() / len(pred)).item())
            loss.backward()
            opt.step()

        valid_loss, valid_accuracy = validate(model, valid_data, loss_fn)

        train_accuracies.append(mean(train_acc))
        train_losses.append(mean(train_loss))
        valid_losses.append(valid_loss)
        valid_accuracies.append(valid_accuracy)
        
        print(f'epoch: {epoch}: train_loss: {mean(train_losses)}, train_acc: {mean(train_acc)}, val_loss: {valid_loss}, val_acc: {valid_accuracy}')
    plot_losses_and_acc(train_losses, train_accuracies, valid_losses, valid_accuracies)
    return model, train_losses, train_accuracies, valid_losses, valid_accuracies

In [61]:
from tqdm import tqdm

In [62]:
model, train_losses, train_accuracies, valid_losses, valid_accuracies = train(model, train_loader, test_loader, loss_fn, opt, epoches=2)

  0%|          | 0/2 [00:01<?, ?it/s] 


OutOfMemoryError: CUDA out of memory. Tried to allocate 392.00 MiB (GPU 0; 4.00 GiB total capacity; 2.69 GiB already allocated; 0 bytes free; 3.12 GiB reserved in total by PyTorch) If reserved memory is >> allocated memory try setting max_split_size_mb to avoid fragmentation.  See documentation for Memory Management and PYTORCH_CUDA_ALLOC_CONF

In [ ]:
torch.cuda.memory_summary(device=None, abbreviated=False)